# Revenue Management

This was an assignment from my Operations Analytics course. The use case here is Cloud Computing.

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

## Cloud Computing

In total, on a given day, Cirrus has 512 CPUs, 1024 GB of memory and 64 GB of GPU memory available. 

Cirrus offers the following instances:


|Instance|Name|CPU (#)|Memory (GB)|GPU (GB)|Price|Rate (#/day)
|-|-|-|-|-|-|-|
|1|C1|16|8|1|\\$7|5.0
|2|C2|32|16|1|\\$12|5.0
|3|C3|64|32|1|\\$24|1.8 
|4|M1|8|32|1|\\$22|3.0
|5|M2|16|64|1|\\$44|2.6
|6|M3|32|128|1|\\$88|1.0
|7|G1|16|16|2|\\$30|0.8 
|8|G2|32|32|6|\\$90|0.4
|9|G3|64|64|8|\\$120|0.3


### Part 1. Capacity control formulation

Let $x_1$, ..., $x_9$ be the number of instances that are reserved of each instance type.

**(a)** The objective function is:

$$
\begin{alignat}{2}
\text{maximize} & \quad & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9\\
\end{alignat}
$$

**(b)** Let $A_{i,j}$ corresponds to how much of resource $i$ is used by instance $j$ on a given day. The contraint on the total memory usage of the instances that are reserved:

$$
\begin{alignat}{2}
& 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \leq 1024 \\
\end{alignat}
$$

**(c)** Since the demand for each type of instance is assumed to follow a Poisson arrival process, and the average arrival rate of instance type 5 is 2.6 per day, the expected number of requests of instance type 5 over the five day period is 13 = (2.6 * 5).

Assuming there will be exactly this many requests over the T = 5 day period, the constraint on the number of requests of instance type 5 we may accept is:

$$
\begin{alignat}{2}
& x_5 \leq 13 \\
\end{alignat}
$$

**(d)** The LP formulation of the T = 5 day capacity control problem is:

$$
\begin{alignat}{2}
& \text{maximize} & 7x_1 + 12x_2 + 24x_3 + 22x_4 + 44x_5 + 88x_6 + 30x_7 + 90x_8 + 120x_9 \\
& \text{subject to} \\
& & 16x_1 + 32x_2 + 64x_3 + 8x_4 + 16x_5 + 32x_6 + 16x_7 + 32x_8 + 64x_9 \leq 512 \\
& & 8x_1 + 16x_2 + 32x_3 + 32x_4 + 64x_5 + 128x_6 + 16x_7 + 32x_8 + 64x_9 \leq 1024 \\
& & x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + 2x_7 + 6x_8 + 8x_9 \leq 64 \\
& & x_1 \leq 25 \\
& & x_2 \leq 25 \\
& & x_3 \leq 9 \\
& & x_4 \leq 15 \\
& & x_5 \leq 13 \\
& & x_6 \leq 5 \\
& & x_7 \leq 4 \\
& & x_8 \leq 2 \\
& & x_9 \leq 1.5 \\
& & x_{i} \geq 0, & \;\; & i \in \{1, ... ,3\} \\ 
\end{alignat}
$$

### Part 2. Solving the capacity control problem in Python/Gurobi

**(a)** The optimal objective value is **$ 1039.43**.

**(b)** In the optimal allocation, approximately **6** requests of instance type C1 are accepted.

In [2]:
# The number of instances
nInstances = 9

# Time horizon:
T = 5

# Price for each instance type:
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])

# Forecasted demand for each instance type: 
rates = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3]) 
instance_demand = rates * T

# Capacity of each resource: 
resource_capacity = np.array([512, 1024, 64])

# Component requirements:
cpu_requirement = np.array([16,32,64,8,16,32,16,32,64])
memory_requirement = np.array([8,16,32,32,64,128,16,32,64])
gpu_requirement = np.array([1,1,1,1,1,1,2,6,8])

In [3]:
from gurobipy import *

# Create the model
m = Model()

# Create decision variables
x_1 = m.addVar(lb = 0, ub = instance_demand[0])
x_2 = m.addVar(lb = 0, ub = instance_demand[1])
x_3 = m.addVar(lb = 0, ub = instance_demand[2])
x_4 = m.addVar(lb = 0, ub = instance_demand[3])
x_5 = m.addVar(lb = 0, ub = instance_demand[4])
x_6 = m.addVar(lb = 0, ub = instance_demand[5])
x_7 = m.addVar(lb = 0, ub = instance_demand[6])
x_8 = m.addVar(lb = 0, ub = instance_demand[7])
x_9 = m.addVar(lb = 0, ub = instance_demand[8])

x = np.array([x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9])

# Create constraints
cpu_constr = m.addConstr( sum( cpu_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[0] )
memory_constr = m.addConstr( sum( memory_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[1] )
gpu_constr = m.addConstr( sum( gpu_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[2] )

# Set the objective
m.setObjective(sum( price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE )

# Update + solve:
m.update()
m.optimize()

# Get the objective value
LP_obj = m.objval

# Get the allocation
allocation = np.array([x_1.x, x_2.x, x_3.x, x_4.x, x_5.x, x_6.x, x_7.x, x_8.x, x_9.x])

# Display the results:
print()
print("Allocation:")
print(allocation) # x_i

print("Objective:")
print(LP_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 9 columns and 27 nonzeros
Model fingerprint: 0x207d4a09
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 2e+01]
  RHS range        [6e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.039428571e+03

Allocation:
[6.28571429 0.         0.         3.42857143 0.         5.
 4.      

### Part 3. Simulating current practice

Currently, Cirrus simply accepts requests in a first-come first-serve fashion, without considering the revenue of the requests.

**(a)** The average number of arrivals of type C1 in the set of simulated sequences is **26.63**.

**(b)** The average number of arrivals, of all types, over the set of simulated sequences are similar to the forecasted # of arrivals based on the Poisson distribution, but they are not exactly the same.

|Instance|Name|Average # of Arrivals|Forecasted # of Arrivals
|-|-|-|-
|1|C1|26.63|25
|2|C2|24.38|25
|3|C3|9.01|9
|4|M1|14.88|15
|5|M2|13.32|13
|6|M3|4.93|5
|7|G1|4.2|4
|8|G2|2.27|2
|9|G3|1.38|1.5

**(c)** The average revenue garnered by Cirrus's current policy (myopic) is **$ 528.28**.

**(d)** The average remaining capacity (of CPUs, memory and GPUs) of this policy is **0.24, 371.52, 37.42**, respectively.

In [4]:
# Preconditions:
# nSimulations = integer specifying number of simulations to run
# rates = array containing arrival rate (# / day) for each of the instance
# types (should be an array with 9 elements)
# T = length of horizon in days

def generateArrivalSequences( nSimulations, rates, T ):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p= rates/total_rate )

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types

In [5]:
# Code to test out above function
np.random.seed(10)
nSimulations_test = 100
rates_test = rates
T_test = 5
arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(nSimulations_test, rates_test, T_test)
arrival_sequences_times[0][:3]

array([0.07414243, 0.1246028 , 0.15928449])

In [6]:
# arrival_sequences_times[11]

In [7]:
# arrival_sequences_types[11]

In [8]:
def calculate_average_arrivals(instance_type):
    nArrivals = 0

    for i in range(nSimulations_test):
        for j in range(len(arrival_sequences_types[i])):
            if arrival_sequences_types[i][j] == instance_type:
                nArrivals += 1
                
    return nArrivals/nSimulations_test

In [9]:
avg_arrivals_C1 = calculate_average_arrivals(0)
print("Average number of arrivals of type C1: ", avg_arrivals_C1)

Average number of arrivals of type C1:  26.63


In [10]:
all_avg_arrivals = [] 

for instance_type in range(nInstances):
    avg_arrivals = calculate_average_arrivals(instance_type)
    all_avg_arrivals.append(avg_arrivals)
    
all_avg_arrivals

[26.63, 24.38, 9.01, 14.88, 13.32, 4.93, 4.2, 2.27, 1.38]

In [11]:
instance_demand

array([25. , 25. ,  9. , 15. , 13. ,  5. ,  4. ,  2. ,  1.5])

In [12]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that simulation
# arrival_sequences_types = array where each entry is sequence of request types for that simulation

np.random.seed(10)
nSimulations = 100
nResources = 3
T = 5
rates = rates
B = resource_capacity
arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(nSimulations, rates, T)


results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy();
    single_revenue = 0.0; # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s]);

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request        
        if ( (b[0] >= cpu_requirement[i]) and (b[1] >= memory_requirement[i]) and (b[2] >= gpu_requirement[i]) ):
            # If there is sufficient capacity, accrue the revenue and remove the capacity
            single_revenue += price[i]
            b[0] -= cpu_requirement[i]
            b[1] -= memory_requirement[i]
            b[2] -= gpu_requirement[i]

    # Save the results of this simulation
    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[0][s] = b[0]
    results_myopic_remaining_capacity[1][s] = b[1]
    results_myopic_remaining_capacity[2][s] = b[2]

# Find the average revenue
mean_myopic_revenue = results_myopic_revenue.mean()
# Find the average remaining quantity of each resource
mean_myopic_remaining_capacity = np.mean(results_myopic_remaining_capacity, axis=1)
    
print("Mean revenue (myopic): ", mean_myopic_revenue)
print("Mean capacity remaining (myopic): ", mean_myopic_remaining_capacity)

Mean revenue (myopic):  528.28
Mean capacity remaining (myopic):  [2.4000e-01 3.7152e+02 3.7420e+01]


### Part 4. A bid-price control policy

**(a)** The approximate opportunity cost of accepting this request for instance type 5 (M2) is **$44**.

**(b)** Suppose that we receive a request at time $t$. The expected number of requests of type $i$ from time $t$ to time $T$ (including both $t$ and $T$) is

$$
\begin{alignat}{2}
& (T - t + 1) \; * \; \text{rate of type} \; i
\end{alignat}
$$

**(c)** The average revenue garnered by the bid-price control policy is **$ 925.59**.

**(d)** The average remaining capacity (of CPUs, memory and GPUs) of this policy is **27.2, 4.88, 20.62**, respectively.

In [13]:
instance_5_opportunity_cost = 16*cpu_constr.pi + 64*memory_constr.pi + 1*gpu_constr.pi
instance_5_opportunity_cost

44.0

In [14]:
# Create the model
m = Model()

m.Params.outputflag = 0

# Create decision variables
x = m.addVars(nInstances)

for i in range(nInstances):
    x[i].ub = instance_demand[i]

# Create constraints
cpu_constr = m.addConstr( sum( cpu_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[0] )
memory_constr = m.addConstr( sum( memory_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[1] )
gpu_constr = m.addConstr( sum( gpu_requirement[i] * x[i] for i in range(nInstances)) <= resource_capacity[2] )

# Set the objective
m.setObjective(sum( price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE )

# Update + solve:
m.update()
m.optimize()

# Get the objective value
LP_obj = m.objval

# Get the allocation
allocation = [ x[i].x for i in range(nInstances)]

# Display the results:
print("Allocation:")
print(allocation) # x_i
print()
print("Objective:")
print(LP_obj)

Allocation:
[6.285714285714286, 0.0, 0.0, 3.428571428571429, 0.0, 5.0, 4.0, 2.0, 1.5]

Objective:
1039.4285714285716


In [15]:
# Preconditions for code below:
# nSimulations = number of simulations to run
# nResources = number of different types of resources (= 3)
# B = numpy array of initial capacities of the resources
# arrival_sequences_times = array where each entry is arrival time sequence for that simulation
# arrival_sequences_types = array where each entry is sequence of request types for that simulation

np.random.seed(10)
nSimulations = 100
nResources = 3
T = 5
rates = rates
B = resource_capacity
arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(nSimulations, rates, T)

def bpc(b, t):
    #for r in range(nResources):
        # Set the RHS of the resource constraint to b[r] here
    cpu_constr.rhs = b[0]
    memory_constr.rhs = b[1]
    gpu_constr.rhs = b[2]

    for i in range(nInstances):
        # Set the RHS of the forecast constraint for each instance
        # type to the expected number of requests over the duration
        # of the remaining horizon (T - t).
        x[i].ub = (T - t) * rates[i]

    # Re-solve the model:
    m.update()
    m.optimize()

    # Obtain the dual values/shadow prices
    dual_val = [cpu_constr.pi, memory_constr.pi, gpu_constr.pi]

    # Return the dual values:
    return dual_val



results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros((nResources, nSimulations))
for s in range(nSimulations):
    b = B.copy() # Initialize the current capacity
    single_revenue = 0.0 # Initialize the revenue garnered in this simulation
    nArrivals = len(arrival_sequences_times[s])
    for j in range(nArrivals):
        # Take the next arrival time and type from the sequence
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is enough capacity
        # if ( (b[0] < cpu_requirement[i]) or (b[1] < memory_requirement[i]) or (b[2] < gpu_requirement[i]) ):
        if ( (b[0] ==0) and (b[1] ==0) and (b[2] ==0) ):
            break
        
        if ( (b[0] >= cpu_requirement[i]) and (b[1] >= memory_requirement[i]) and (b[2] >= gpu_requirement[i]) ):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)

            # Check if the revenue is at least the sum of the bid prices
            bid_price = cpu_requirement[i]*dual_val[0] + memory_requirement[i]*dual_val[1] + gpu_requirement[i]*dual_val[2]
            if ( price[i] >= bid_price):
                # If there is sufficient capacity, accrue the revenue and remove the capacity
                single_revenue += price[i]
                b[0] -= cpu_requirement[i]
                b[1] -= memory_requirement[i]
                b[2] -= gpu_requirement[i]

    # Save the results of this simulation here:
    results_revenue[s] = single_revenue
    results_remaining_capacity[0][s] = b[0]
    results_remaining_capacity[1][s] = b[1]
    results_remaining_capacity[2][s] = b[2]

# Find the average revenue
mean_revenue = results_revenue.mean()
# Find the average remaining quantity of each resource
mean_remaining_capacity = np.mean(results_remaining_capacity, axis=1)
    
print("Mean revenue (bid-price control): ", mean_revenue)
print("Mean capacity remaining (bid-price control): ", mean_remaining_capacity)

Mean revenue (bid-price control):  925.59
Mean capacity remaining (bid-price control):  [27.2   4.88 20.62]


In [42]:
# EOF